In [2]:
from geoplot import utils
import geoplot as gplt
from geoplot import crs as gcrs
import geopandas as gpd
import pandas as pd
import shapely
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
savefig_kwargs = {'bbox_inches': 'tight', 'pad_inches': 0.25}

In [4]:
!mkdir ../figures/pointplot
!mkdir ../figures/kdeplot
!mkdir ../figures/polyplot
!mkdir ../figures/cartogram
!mkdir ../figures/aggplot
!mkdir ../figures/sankey
!mkdir ../figures/choropleth

# Load example data

In [3]:
import quilt
quilt.install('ResidentMario/geoplot_data')

ResidentMario/geoplot_data already installed.
Overwrite? (y/n) y
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.
Fragment already installed; skipping.


In [4]:
from quilt.data.ResidentMario import geoplot_data
import geopandas as gpd

cities = gpd.read_file(geoplot_data.usa_cities())
boroughs = gpd.read_file(geoplot_data.nyc_boroughs())
dc = gpd.read_file(geoplot_data.dc_roads())
census_tracts = gpd.read_file(geoplot_data.nyc_census_tracts())

In [11]:
quilt.install('ResidentMario/nyc_collisions')

100%|██████████| 6.56M/6.56M [00:06<00:00, 1.02MB/s]


In [4]:
from quilt.data.ResidentMario import nyc_collisions
collisions = nyc_collisions.collisions()

def pointify(srs):
    try:
        lat, long = float(srs['LATITUDE']), float(srs['LONGITUDE'])
        if pd.isnull(lat) or pd.isnull(long):
            return shapely.geometry.Point((0, 0))
        else:
            return shapely.geometry.Point((long, lat))
    except ValueError:
        return shapely.geometry.Point((0, 0))

collisions = gpd.GeoDataFrame(collisions.head(5000), geometry=collisions.head(5000).apply(pointify, axis='columns'))
collisions = collisions[collisions.geometry.map(lambda srs: not (srs.x == 0))]
collisions['BOROUGH'] = collisions['BOROUGH'].str.strip().map(lambda v: np.nan if len(v) == 0 else v)

# Pointplot

In [5]:
points = utils.gaussian_points()
gplt.pointplot(points, projection=gcrs.PlateCarree())
plt.savefig('../figures/pointplot/pointplot-initial.png', **savefig_kwargs)

In [13]:
gplt.pointplot(cities, projection=gcrs.AlbersEqualArea(), hue='ELEV_IN_FT')
plt.savefig('../figures/pointplot/pointplot-hue.png', **savefig_kwargs)

In [14]:
gplt.pointplot(cities, projection=gcrs.AlbersEqualArea(), hue='ELEV_IN_FT', legend=True)
plt.savefig('../figures/pointplot/pointplot-legend.png', **savefig_kwargs)

In [15]:
gplt.pointplot(cities, projection=gcrs.AlbersEqualArea(), hue='ELEV_IN_FT', legend=True, legend_labels=list('ABCDE'))
plt.savefig('../figures/pointplot/pointplot-legend-labels.png', **savefig_kwargs)

In [16]:
gplt.pointplot(cities, projection=gcrs.AlbersEqualArea(), hue='ELEV_IN_FT', legend=True, scheme='equal_interval')
plt.savefig('../figures/pointplot/pointplot-scheme.png', **savefig_kwargs)

In [42]:
gplt.pointplot(collisions[collisions['BOROUGH'].notnull()], projection=gcrs.AlbersEqualArea(), hue='BOROUGH',
               legend=True, categorical=True)
plt.savefig('../figures/pointplot/pointplot-categorical.png', **savefig_kwargs)

In [43]:
gplt.pointplot(collisions[collisions['BOROUGH'].notnull()], projection=gcrs.AlbersEqualArea(), 
               hue='BOROUGH', categorical=True,
               legend=True, legend_kwargs={'loc': 'upper left'})
plt.savefig('../figures/pointplot/pointplot-legend-kwargs.png', **savefig_kwargs)

In [44]:
gplt.pointplot(collisions[collisions['BOROUGH'].notnull()], projection=gcrs.AlbersEqualArea(), 
               hue='BOROUGH', categorical=True, 
               legend=True, legend_kwargs={'loc': 'upper left'},
               edgecolor='white', linewidth=0.5)
plt.savefig('../figures/pointplot/pointplot-kwargs.png', **savefig_kwargs)

In [2]:
more_points = utils.gaussian_points(n=3000)
data = gpd.GeoDataFrame(data={'classification': utils.classify_clusters(more_points, n=6)}, geometry=more_points)
data = data.assign(var=data['classification'] * 0.5 + pd.Series([p.x + p.y for p in data['geometry']]) + 50 + np.random.normal(size=3000) * 8)

In [7]:
gplt.pointplot(data, projection=gcrs.AlbersEqualArea(),
               hue='var', k=8,
               edgecolor='white', linewidth=0.5,
               legend=True, legend_kwargs={'bbox_to_anchor': (1.15, 1.0)})
plt.savefig('../figures/pointplot/pointplot-k.png', **savefig_kwargs)

In [8]:
more_points = utils.gaussian_points(n=3000)
data = gpd.GeoDataFrame(data={'classification': utils.classify_clusters(more_points, n=6)}, geometry=more_points)
data = data.assign(var=data['classification'] * 0.5 + pd.Series([p.x + p.y for p in data['geometry']]) + 50 + np.random.normal(size=3000) * 8)
gplt.pointplot(data, projection=gcrs.AlbersEqualArea(),
               hue='var', cmap='inferno', k=8,
               edgecolor='white', linewidth=0.5,
               legend=True, legend_kwargs={'bbox_to_anchor': (1.15, 1.0)})
plt.savefig('../figures/pointplot/pointplot-cmap.png', **savefig_kwargs)

In [9]:
more_points = utils.gaussian_points(n=3000)
data = gpd.GeoDataFrame(data={'classification': utils.classify_clusters(more_points, n=6)}, geometry=more_points)
data = data.assign(var=data['classification'] * 0.5 + pd.Series([p.x + p.y for p in data['geometry']]) + 50 + np.random.normal(size=3000) * 8)
gplt.pointplot(data, projection=gcrs.AlbersEqualArea(),
               hue='var', cmap='inferno', k=None,
               edgecolor='white', linewidth=0.5,
               legend=True)
plt.savefig('../figures/pointplot/pointplot-k-none.png', **savefig_kwargs)

In [45]:
gplt.pointplot(collisions, projection=gcrs.AlbersEqualArea(), 
               scale='NUMBER OF PERSONS INJURED',
               legend=True, legend_kwargs={'loc': 'upper left'})
plt.savefig('../figures/pointplot/pointplot-scale.png', **savefig_kwargs)

In [46]:
gplt.pointplot(collisions, projection=gcrs.AlbersEqualArea(), 
               scale='NUMBER OF PERSONS INJURED', limits=(0, 10),
               legend=True, legend_kwargs={'loc': 'upper left'})
plt.savefig('../figures/pointplot/pointplot-limits.png', **savefig_kwargs)

In [47]:
def trivial_scale(minval, maxval):
    def scalar(val):
        return 2
    return scalar

gplt.pointplot(collisions, projection=gcrs.AlbersEqualArea(), 
               scale='NUMBER OF PERSONS INJURED', scale_func=trivial_scale,
               legend=True, legend_kwargs={'loc': 'upper left'})
plt.savefig('../figures/pointplot/pointplot-scale-func.png', **savefig_kwargs)

In [48]:
gplt.pointplot(collisions[collisions['BOROUGH'].notnull()],
               projection=gcrs.AlbersEqualArea(),
               hue='BOROUGH', categorical=True,
               scale='NUMBER OF PERSONS INJURED', limits=(0, 10),
               legend=True, legend_kwargs={'loc': 'upper left'})
plt.savefig('../figures/pointplot/pointplot-hue-scale.png', **savefig_kwargs)

In [49]:
gplt.pointplot(collisions[collisions['BOROUGH'].notnull()],
               projection=gcrs.AlbersEqualArea(),
               hue='BOROUGH', categorical=True,
               scale='NUMBER OF PERSONS INJURED', limits=(0, 10),
               legend=True, legend_kwargs={'loc': 'upper left'},
               legend_var='scale')
plt.savefig('../figures/pointplot/pointplot-legend-var.png', **savefig_kwargs)

# Choropleth

In [12]:
from random import randint
polygons = utils.gaussian_polygons(n=20, points=utils.gaussian_points(n=3000))
polydata = gpd.GeoDataFrame(data={'classification': [randint(1, 20) for _ in range(len(polygons))]}, geometry=polygons)

In [13]:
polydata['latdep'] = polydata['classification'] + np.random.normal() * 5 + 5 + polydata.geometry.envelope.exterior.map(lambda ext: ext.coords[0][1] + ext.coords[0][0])

In [14]:
gplt.choropleth(polydata, hue='latdep', projection=gcrs.PlateCarree())
plt.savefig('../figures/choropleth/choropleth-initial.png', **savefig_kwargs)

In [15]:
gplt.choropleth(polydata, hue='latdep', projection=gcrs.PlateCarree(), cmap='Blues')
plt.savefig('../figures/choropleth/choropleth-cmap.png', **savefig_kwargs)

In [16]:
gplt.choropleth(boroughs, projection=gcrs.AlbersEqualArea(), hue='BoroName', categorical=True)
plt.savefig('../figures/choropleth/choropleth-categorical.png', **savefig_kwargs)

In [17]:
gplt.choropleth(boroughs, projection=gcrs.AlbersEqualArea(), hue='BoroName', categorical=True,
                legend=True)
plt.savefig('../figures/choropleth/choropleth-legend.png', **savefig_kwargs)

In [18]:
gplt.choropleth(boroughs, projection=gcrs.AlbersEqualArea(), hue='BoroName', categorical=True,
                legend=True, legend_kwargs={'loc': 'upper left'})
plt.savefig('../figures/choropleth/choropleth-legend-kwargs.png', **savefig_kwargs)

In [19]:
gplt.choropleth(boroughs, projection=gcrs.AlbersEqualArea(), hue='BoroName', categorical=True,
                linewidth=0)
plt.savefig('../figures/choropleth/choropleth-kwargs.png', **savefig_kwargs)

In [5]:
from random import randint
census_tracts['mock_data'] = [randint(10, 1000) for _ in range(len(census_tracts))]
census_tracts['mock_category'] = [randint(1, 5) for _ in range(len(census_tracts))]

In [6]:
gplt.choropleth(census_tracts, hue='mock_data', projection=gcrs.AlbersEqualArea(),
                legend=True, edgecolor='white', linewidth=0.5, legend_kwargs={'loc': 'upper left'},
                k=2)
plt.savefig('../figures/choropleth/choropleth-k.png', **savefig_kwargs)

In [7]:
gplt.choropleth(census_tracts, hue='mock_data', projection=gcrs.AlbersEqualArea(),
                legend=True, edgecolor='white', linewidth=0.5, cmap='viridis',
                k=None)
plt.savefig('../figures/choropleth/choropleth-k-none.png', **savefig_kwargs)

In [8]:
gplt.choropleth(census_tracts, hue='mock_data', projection=gcrs.AlbersEqualArea(),
                edgecolor='white', linewidth=0.5,
                legend=True, legend_kwargs={'loc': 'upper left'},
                legend_labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
plt.savefig('../figures/choropleth/choropleth-legend-labels.png', **savefig_kwargs)

In [9]:
gplt.choropleth(census_tracts, hue='mock_data', projection=gcrs.AlbersEqualArea(),
                legend=True, edgecolor='white', linewidth=0.5, legend_kwargs={'loc': 'upper left'},
                scheme='equal_interval')
plt.savefig('../figures/choropleth/choropleth-scheme.png', **savefig_kwargs)

# Polyplot

In [20]:
gplt.polyplot(boroughs, projection=gcrs.AlbersEqualArea())
plt.savefig('../figures/polyplot/polyplot-initial.png', **savefig_kwargs)

In [21]:
gplt.polyplot(boroughs, projection=gcrs.AlbersEqualArea(), 
              linewidth=0, facecolor='lightgray')
plt.savefig('../figures/polyplot/polyplot-kwargs.png', **savefig_kwargs)

In [50]:
ax = gplt.polyplot(boroughs, projection=gcrs.AlbersEqualArea())
gplt.pointplot(collisions[collisions['BOROUGH'].notnull()], projection=gcrs.AlbersEqualArea(), 
               hue='BOROUGH', categorical=True, 
               legend=True, edgecolor='white', linewidth=0.5, legend_kwargs={'loc': 'upper left'},
               ax=ax)
plt.savefig('../figures/polyplot/polyplot-stacked.png', **savefig_kwargs)

# Aggplot

In [5]:
from shapely.geometry import Point

collisions['LATITUDE'] = collisions['LATITUDE'].astype(float)
collisions['LONGITUDE'] = collisions['LONGITUDE'].astype(float)

latmax, latmin, longmax, longmin = collisions['LATITUDE'].max(), collisions['LATITUDE'].min(),\
                                   collisions['LONGITUDE'].max(), collisions['LONGITUDE'].min()
    
def genpattern(srs):
    return (srs['LATITUDE'] - latmin) / (latmax - latmin)

collisions['LATDEP'] = collisions.apply(genpattern, axis='columns')

In [55]:
gplt.aggplot(collisions, projection=gcrs.PlateCarree(), hue='LATDEP')
plt.savefig('../figures/aggplot/aggplot-initial.png', **savefig_kwargs)

In [57]:
gplt.aggplot(collisions, nmin=10, nmax=250, nsig=5, projection=gcrs.PlateCarree(), hue='LATDEP', cmap='Blues')
plt.savefig('../figures/aggplot/aggplot-quadtree-tuned.png', **savefig_kwargs)

In [58]:
gplt.aggplot(collisions, projection=gcrs.PlateCarree(), hue='NUMBER OF PERSONS INJURED', cmap='Reds',
             by='BOROUGH')
plt.savefig('../figures/aggplot/aggplot-hulls.png', **savefig_kwargs)

In [59]:
boroughs_2 = boroughs.set_index('BoroName')
boroughs_2.index = [b.upper() for b in boroughs_2.index]
gplt.aggplot(collisions, projection=gcrs.PlateCarree(), hue='NUMBER OF PERSONS INJURED', cmap='Reds',
             geometry=boroughs_2, by='BOROUGH')
plt.savefig('../figures/aggplot/aggplot-by.png', **savefig_kwargs)

In [60]:
boroughs_2 = boroughs.set_index('BoroName')
boroughs_2.index = [b.upper() for b in boroughs_2.index]
gplt.aggplot(collisions, projection=gcrs.PlateCarree(), hue='NUMBER OF PERSONS INJURED', cmap='Reds',
             geometry=boroughs_2, by='BOROUGH', agg=len)
plt.savefig('../figures/aggplot/aggplot-agg.png', **savefig_kwargs)

In [61]:
gplt.aggplot(collisions, projection=gcrs.PlateCarree(), hue='NUMBER OF PERSONS INJURED', cmap='Reds',
             geometry=boroughs_2, by='BOROUGH', agg=len, legend=False)
plt.savefig('../figures/aggplot/aggplot-legend.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [62]:
gplt.aggplot(collisions, projection=gcrs.PlateCarree(), hue='NUMBER OF PERSONS INJURED', cmap='Reds',
             geometry=boroughs_2, by='BOROUGH', agg=len, linewidth=0)
plt.savefig('../figures/aggplot/aggplot-kwargs.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [63]:
gplt.aggplot(collisions, projection=gcrs.PlateCarree(), hue='NUMBER OF PERSONS INJURED', cmap='Reds',
             geometry=boroughs_2, by='BOROUGH', agg=len, linewidth=0, 
             legend_kwargs={'orientation': 'horizontal'})
plt.savefig('../figures/aggplot/aggplot-legend-kwargs.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


# Cartogram

In [24]:
boroughs['Population'] = [378, 1951, 2300, 1487, 1203]
boroughs['Population Density'] = boroughs['Population'] / boroughs['Shape_Area']

In [25]:
gplt.cartogram(boroughs, scale='Population Density', projection=gcrs.AlbersEqualArea())
plt.savefig('../figures/cartogram/cartogram-initial.png', **savefig_kwargs)

In [26]:
gplt.cartogram(boroughs, scale='Population Density', projection=gcrs.AlbersEqualArea(),
               trace=False, legend=True)
plt.savefig('../figures/cartogram/cartogram-trace-legend.png', **savefig_kwargs)

In [27]:
gplt.cartogram(boroughs, scale='Population Density', projection=gcrs.AlbersEqualArea(),
               trace=False, legend=True, legend_kwargs={'loc': 'upper left'})
plt.savefig('../figures/cartogram/cartogram-legend-kwargs.png', **savefig_kwargs)

In [28]:
gplt.cartogram(boroughs, scale='Population Density', projection=gcrs.AlbersEqualArea(), legend=True,
               legend_values=[2.32779655e-07, 6.39683197e-07, 1.01364661e-06, 1.17380941e-06, 2.33642596e-06][::-1],
               legend_labels=['Manhattan', 'Brooklyn', 'Queens', 'The Bronx', 'Staten Island'],
               legend_kwargs={'loc': 'upper left'})
plt.savefig('../figures/cartogram/cartogram-legend-labels.png', **savefig_kwargs)

In [29]:
gplt.cartogram(boroughs, scale='Population Density', projection=gcrs.AlbersEqualArea(),
               edgecolor='darkgreen')
plt.savefig('../figures/cartogram/cartogram-kwargs.png', **savefig_kwargs)

In [30]:
gplt.cartogram(boroughs, scale='Population Density', projection=gcrs.AlbersEqualArea(),
               trace_kwargs={'edgecolor': 'lightgreen'})
plt.savefig('../figures/cartogram/cartogram-trace-kwargs.png', **savefig_kwargs)

In [31]:
gplt.cartogram(boroughs, scale='Population Density', projection=gcrs.AlbersEqualArea(),
               limits=(0.5, 1))
plt.savefig('../figures/cartogram/cartogram-limits.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [32]:
def trivial_scale(minval, maxval):
    def scalar(val):
        return 0.5
    return scalar

gplt.cartogram(boroughs, scale='Population Density', projection=gcrs.AlbersEqualArea(),
               limits=(0.5, 1), scale_func=trivial_scale)
plt.savefig('../figures/cartogram/cartogram-scale-func.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [33]:
gplt.cartogram(boroughs, scale='Population Density', projection=gcrs.AlbersEqualArea(),
               hue='Population Density', k=None, cmap='Blues')
plt.savefig('../figures/cartogram/cartogram-hue.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## KDEPlot

In [64]:
ax = gplt.kdeplot(collisions, projection=gcrs.AlbersEqualArea())
plt.savefig('../figures/kdeplot/kdeplot-initial.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [65]:
ax = gplt.kdeplot(collisions, projection=gcrs.AlbersEqualArea())
gplt.polyplot(boroughs, projection=gcrs.AlbersEqualArea(), ax=ax)
plt.savefig('../figures/kdeplot/kdeplot-overlay.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [66]:
ax = gplt.kdeplot(collisions, projection=gcrs.AlbersEqualArea(), shade=True)
gplt.polyplot(boroughs, projection=gcrs.AlbersEqualArea(), ax=ax)
plt.savefig('../figures/kdeplot/kdeplot-shade.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [67]:
ax = gplt.kdeplot(collisions, projection=gcrs.AlbersEqualArea(), n_levels=30)
gplt.polyplot(boroughs, projection=gcrs.AlbersEqualArea(), ax=ax)
plt.savefig('../figures/kdeplot/kdeplot-n-levels.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [68]:
ax = gplt.kdeplot(collisions, projection=gcrs.AlbersEqualArea(), cmap='Purples')
gplt.polyplot(boroughs, projection=gcrs.AlbersEqualArea(), ax=ax)
plt.savefig('../figures/kdeplot/kdeplot-cmap.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [69]:
ax = gplt.kdeplot(collisions, projection=gcrs.AlbersEqualArea(),
                  shade=True, clip=boroughs.geometry)
gplt.polyplot(boroughs, projection=gcrs.AlbersEqualArea(), ax=ax)
plt.savefig('../figures/kdeplot/kdeplot-clip.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## Sankey

In [35]:
from shapely.geometry import Point
starting_points = gpd.GeoSeries([Point(-90, 45), Point(20, 45)])
ending_points = gpd.GeoSeries([Point(45, 45), Point(35, 0)])
mock_data = pd.DataFrame(data={'origin': starting_points, 'destination': ending_points})

In [36]:
gplt.sankey(mock_data, start='origin', end='destination', projection=gcrs.PlateCarree())
plt.savefig('../figures/sankey/sankey-initial.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [37]:
ax = gplt.sankey(mock_data, start='origin', end='destination', projection=gcrs.PlateCarree())
ax.coastlines()
plt.savefig('../figures/sankey/sankey-geospatial-context.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [38]:
network = utils.uniform_random_global_network()

In [39]:
ax = gplt.sankey(projection=gcrs.PlateCarree(), start=network['from'], end=network['to'])
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-alternative-method-signature.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [40]:
ax = gplt.sankey(projection=gcrs.Orthographic(), start=network['from'], end=network['to'],
                 extent=(-180, 180, -90, 90))
ax.set_global()
ax.coastlines()
ax.outline_patch.set_visible(True)
plt.savefig('../figures/sankey/sankey-greatest-circle-distance.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [41]:
from cartopy.crs import PlateCarree

ax = gplt.sankey(projection=gcrs.PlateCarree(), start=network['from'], end=network['to'],
                 path=PlateCarree())
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-path-projection.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [45]:
gplt.sankey(dc, path=dc.geometry, projection=gcrs.AlbersEqualArea(), scale='aadt',
            limits=(0.1, 10))
plt.savefig('../figures/sankey/sankey-path.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [46]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 hue='mock_variable')
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-hue.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [47]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 hue='mock_variable', cmap='RdYlBu')
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-cmap.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [48]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 hue='mock_variable', cmap='RdYlBu',
                 legend=True)
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-legend.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [49]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 hue='mock_variable', cmap='RdYlBu',
                 legend=True, legend_kwargs={'bbox_to_anchor': (1.1, 1.0)})
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-legend-kwargs.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [50]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 hue='mock_variable', cmap='RdYlBu',
                 legend=True, legend_kwargs={'bbox_to_anchor': (1.1, 1.0)},
                 legend_labels=['Very Low', 'Low', 'Average', 'High', 'Very High'])
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-legend-labels.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [51]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 hue='mock_variable', cmap='RdYlBu',
                 legend=True, legend_kwargs={'bbox_to_anchor': (1.1, 1.0)},
                 k=3)
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-k.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [52]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 hue='mock_variable', cmap='RdYlBu',
                 legend=True, legend_kwargs={'bbox_to_anchor': (1.1, 1.0)},
                 k=3, scheme='equal_interval')
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-scheme.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [53]:
network['above_meridian'] = network['from'].map(lambda point: point.y > 0)

In [54]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 hue='above_meridian', cmap='RdYlBu',
                 legend=True, legend_kwargs={'bbox_to_anchor': (1.1, 1.0)},
                 categorical=True)
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-categorical.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [55]:
network['mock_data'] = [np.random.normal(100, 50) for _ in range(len(network))]

In [56]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 scale='mock_data',
                 legend=True, legend_kwargs={'bbox_to_anchor': (1.1, 1.0)},
                 color='lightblue')
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-scale.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [57]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 scale='mock_data', limits=(1, 3),
                 legend=True, legend_kwargs={'bbox_to_anchor': (1.1, 1.0)},
                 color='lightblue')
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-limits.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [58]:
def trivial_scale(minval, maxval):
    def scalar(val):
        return 2
    return scalar

ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 scale='mock_data', scale_func=trivial_scale,
                 legend=True, legend_kwargs={'bbox_to_anchor': (1.1, 1.0)},
                 color='lightblue')
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-scale-func.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [59]:
ax = gplt.sankey(network, projection=gcrs.PlateCarree(),
                 start='from', end='to',
                 scale='mock_data',
                 legend=True, legend_kwargs={'bbox_to_anchor': (1.1, 1.0)},
                 hue='mock_data', legend_var="hue")
ax.set_global()
ax.coastlines()
plt.savefig('../figures/sankey/sankey-legend-var.png', **savefig_kwargs)

/Users/Honors/anaconda/envs/geoplot/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
